# Machine learning (ML) and committee models (CMs)

## Import libraries

In [ ]:
import xarray as xr
import fsspec
import netCDF4 as nc

import matplotlib.pyplot as plt
from esmtools.stats import autocorr, corr
from scipy.signal import correlate2d
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

## Open the data

In [ ]:
hindcast_observation = xr.open_zarr('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\hindcast-like-observations_2000-2019_biweekly_deterministic.zarr')
obs00_19_ter = xr.open_zarr('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\hindcast-like-observations_2000-2019_biweekly_terciled.zarr')
tercile_edges = xr.open_dataset('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\hindcast-like-observations_2000-2019_biweekly_tercile-edges.nc')
hindcast_ecmwf = xr.open_zarr('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr')
forecast_ecmwf = xr.open_zarr('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\ecmwf_forecast-input_2020_biweekly_deterministic.zarr')

In [ ]:
forecast_observation = xr.open_zarr('D:/Downloads/s2s-ai-challenge-template-hydroinformatics-group-master-data/data/forecast-like-observations_2020_biweekly_deterministic.zarr')

In [ ]:
tercile_forecast = nc.Dataset("D:\\Downloads\\s2s-ai-challenge-template-hydroinformatics-group-master-data\\data\\forecast-like-observations_2020_biweekly_terciled.nc")
ter_forecast = xr.open_dataset(xr.backends.NetCDF4DataStore(tercile_forecast))

### Add classes

In [ ]:
def create_classes(dataset, observation, tercile_edges, variable, lead_time, start, end):
    
    x = dataset[variable].sel(forecast_time= slice(start, end)).isel(lead_time = lead_time)
    
    upper = []
    lower = []
    for i in range(0,len(dataset.forecast_time)):
        if i < 53:
            u = tercile_edges[variable].isel(week = i, category_edge = 1, lead_time = lead_time).values
            upper.append(u)
            l = tercile_edges[variable].isel(week = i, category_edge = 0, lead_time = lead_time).values
            lower.append(l)

    for i in range(0,(int(len(dataset.forecast_time)/53)-1)):
        for j in range(0, 53):
            upper.append(upper[j])
            lower.append(lower[j])
            
    dataset['upper']=(['forecast_time', 'latitude', 'longitude'], upper)
    dataset['lower']=(['forecast_time','latitude', 'longitude'], lower)

    u = dataset.upper.sel(forecast_time= slice(start, end))

    l = dataset.lower.sel(forecast_time= slice(start, end))
    
    forecast_time = x.forecast_time
    
    classes = []
    
    for i in range(0, len(forecast_time)):

        above = (dataset[variable].isel(forecast_time = i) * 0 + 2).where(dataset[variable].isel(forecast_time = i) - dataset.upper.isel(forecast_time = i) > 0, 0)
        nnormal = (dataset[variable].isel(forecast_time = i) * 0 + 1).where(dataset[variable].isel(forecast_time = i) - dataset.upper.isel(forecast_time = i) <= 0, 0).where(dataset[variable].isel(forecast_time = i) - dataset.lower.isel(forecast_time = i) >= 0, 0)
        below = (dataset[variable].isel(forecast_time = i) * 0 ).where(dataset[variable].isel(forecast_time = i) - dataset.lower.isel(forecast_time = i) < 0, 0)

        c = above + nnormal + below
#         c = c.where(np.isfinite(observation.tp.isel(forecast_time=i, lead_time=0))==1)

        classes.append(c)
#         
        
#     dataset['classes']=(['forecast_time'], classes)
#     dataset['classes'] = ([], classes)
    combined = xr.concat(classes, dim = 'forecast_time')
    cl = combined.where(np.isfinite(observation.tp.isel(forecast_time=0, lead_time=0))==1)
#     dataset['cl'] = cl
    del dataset['upper']
    del dataset['lower']
    return cl

In [ ]:
target_class = create_classes(hindcast_observation, hindcast_observation, tercile_edges, 'tp', 0, '2000-01-01', '2019-12-30')

## K-nearest neighbors algorithm (K-NN)

In [ ]:
# Build the knn model

# in this code the selected data is fixed to the first 18 years for training and the last two years for testing

# import the data

# hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
hobs_int = hobs.interpolate_na(dim = 'forecast_time')
hecm_int = hecm.interpolate_na(dim = 'forecast_time')

target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

# modify the longitude and latitude to the selected regions:

# regions

region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

# longitude = region1.longitude
# latitude  = region1.latitude

# longitude = region2.longitude
# latitude  = region2.latitude

longitude = region3.longitude
latitude  = region3.latitude

# Empty matrices for the results

x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

training_acc = np.zeros((1, len(latitude), len(longitude)))
testing_acc = np.zeros((1, len(latitude), len(longitude)))

i = 0
for long in longitude:
    
    j = 0
    for lat in latitude:
        
        # choose the input and output data
        
        # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
        hecm_cell = hecm.sel(longitude = long, latitude = lat)
        hobs_cell = hobs.sel(longitude = long, latitude = lat)
        hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
        hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
        # hobs_int['cl'] = target_cl
        
        # operations
        # x inputs
        
        # training data

        x = hobs_int.sel(forecast_time = slice('2000-01-01', '2012-12-30'))
        xx = x.to_dataframe().reset_index()
        xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
        xx_inpVar = xx_inp[['tp','t2m']]
        x_train = xx_inpVar.values

        y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2012-12-30'))
        yy = y.to_dataframe().reset_index()
        yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
        yy_inpVar = yy_inp[['tp']]
        y_train = yy_inpVar.values.ravel()

        # testing data

        x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))
        xx = x.to_dataframe().reset_index()
        xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
        xx_inpVar = xx_inp[['tp','t2m']]
        x_test = xx_inpVar.values

        y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2018-01-01', '2019-12-30'))
        yy = y.to_dataframe().reset_index()
        yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
        yy_inpVar = yy_inp[['tp']]
        y_test = yy_inpVar.values.ravel()

        # Building the Knn model
        
        knn = KNeighborsClassifier(n_neighbors= 3)
        
        knn_model = knn.fit(x_train, y_train)

        training_accuracy = knn_model.score(x_train, y_train)
        
        # Knn predictions

        yy_pred = knn.predict_proba(x_test)
        accuracy = knn_model.score(x_test, y_test)
        
        # add the values to the empty array
        
        prediction_an[:, j, i] = yy_pred[:,2]
        prediction_nn[:, j, i] = yy_pred[:,1]
        prediction_bn[:, j, i] = yy_pred[:,0]
        
        testing[:, j, i] = y_test
        
        training_acc[0, j, i] = training_accuracy 
        testing_acc[0, j, i] = accuracy   
        
        j = j + 1 
    
        print(1)
        
    i = i + 1

# Make an xarray of predictions data

time_step = [i for i in range(0, len(yy_pred))]

longitude = longitude.values
latitude = latitude.values

predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),      
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions_an",
    ),
)

predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),      
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions_nn",
    ),
)

predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),      
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions_bn",
    ),
)

# Make an xarray of tests data

test = xr.DataArray(
    data=testing,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),       
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions",
    ),
)

# make categories from the predictions

categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])

categorical_p = categories.rename({'time_step':'forecast_time'})

# Make an xarray for the testing accuracies

test_acc = xr.DataArray(
    data=testing_acc,
    dims=["training_accuracy", "latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),       
    ),
    attrs=dict(
        description="testing accuracy",
    ),
)

# Make an xarray for the training accuracies

train_acc = xr.DataArray(
    data=training_acc,
    dims=["training_accuracy", "latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),       
    ),
    attrs=dict(
        description="testing accuracy",
    ),
)

In [ ]:
hindcast_observation = xr.open_zarr('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\hindcast-like-observations_2000-2019_biweekly_deterministic.zarr')
hindcast_ecmwf = xr.open_zarr('D:\Downloads\s2s-ai-challenge-template-hydroinformatics-group-master-data\data\ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr')

In [ ]:
# Build the knn model

# in this code the selected data is fixed to the first 18 years for training and the last two years for testing

# adding the spatial term

# import the data

# hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
hobs_int = hobs.interpolate_na(dim = 'forecast_time')
hecm_int = hecm.interpolate_na(dim = 'forecast_time')

target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

# modify the longitude and latitude to the selected regions:

# regions

region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

# longitude = region1.longitude
# latitude  = region1.latitude

# longitude = region2.longitude
# latitude  = region2.latitude

longitude = region3.longitude
latitude  = region3.latitude

# Empty matrices for the results

x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

training_acc = np.zeros((1, len(latitude), len(longitude)))
testing_acc = np.zeros((1, len(latitude), len(longitude)))

i = 0
for long in longitude:
    
    j = 0
    for lat in latitude:
        
        # choose the input and output data
        
        # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
        hecm_cell = hecm.sel(longitude = long, latitude = lat)
        hobs_cell = hobs.sel(longitude = long, latitude = lat)
        hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
        hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
        # hobs_int['cl'] = target_cl
        
        # operations
        # x inputs
        
        # training data
        x = hobs_int.sel(forecast_time = slice('2000-01-01', '2012-12-30'))
        xx = x.to_dataframe().reset_index()
        
        # spatially correlated inputs
#         spatial = 'one'
#         spatial = 'two'
        spatial = 'three'

        # add the correlated inputs
        xx['tp_lag1'] = xx.tp.shift(1).bfill()

        if spatial == 'one':
            disla = lat - (1 * 1.5)
            dislo = long - (1 * 1.5)
            dla = np.arange(0, 4.5, 1.5)
            dlo = np.arange(0, 4.5, 1.5)
            v = 'var 4'

        if spatial == 'two':
            disla = lat - (2 * 1.5)
            dislo = long - (2 * 1.5)
            dla = np.arange(0, 7.5, 1.5)
            dlo = np.arange(0, 7.5, 1.5)
            v = 'var 12'

        if spatial == 'three':
            disla = lat - (3 * 1.5)
            dislo = long - (3 * 1.5)
            dla = np.arange(0, 10.5, 1.5)
            dlo = np.arange(0, 10.5, 1.5)
            v = 'var 24'

        # var = []
        nam = 0
        for ii in dlo:
            for jj in dla:

                xx['Var'] = hobs.tp.sel(latitude = disla + jj, longitude = dislo + ii).sel(forecast_time = slice('2000-01-01', '2012-12-30'))
                name = 'var {}'.format(nam)
                xx.rename(columns={'Var': name}, inplace=True)
                nam = nam + 1

        x_dropna = xx.dropna(how='all', axis=1)
        x_inp = x_dropna.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
        x_inpVar = x_inp.drop([v], axis = 'columns')

        x_train = x_inpVar.values

        y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2012-12-30'))
        yy = y.to_dataframe().reset_index()
        yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
        yy_inpVar = yy_inp[['tp']]
        y_train = yy_inpVar.values.ravel()

        # testing data
        x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))
        xx = x.to_dataframe().reset_index()

        # add the correlated inputs
        xx['tp_lag1'] = xx.tp.shift(1).bfill()

        nam = 0
        for ii in dlo:
            for jj in dla:

                xx['Var'] = hecm.tp.sel(latitude = disla + jj, longitude = dislo + ii).sel(forecast_time = slice('2018-01-01', '2019-12-30')).mean(dim = 'realization')
                name = 'var {}'.format(nam)
                xx.rename(columns={'Var': name}, inplace=True)
                nam = nam + 1

        xx_dropna = xx.dropna(how='all', axis=1)
        xx_inpVar = xx_dropna[x_inpVar.columns]

        x_test = xx_inpVar.values

        y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2018-01-01', '2019-12-30'))
        yy = y.to_dataframe().reset_index()
        yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
        yy_inpVar = yy_inp[['tp']]
        y_test = yy_inpVar.values.ravel()


        # Building the Knn model
        
        knn = KNeighborsClassifier(n_neighbors= 3)
        
        knn_model = knn.fit(x_train, y_train)

        training_accuracy = knn_model.score(x_train, y_train)
        
        # Knn predictions

        yy_pred = knn.predict_proba(x_test)
        accuracy = knn_model.score(x_test, y_test)
        
        # add the values to the empty array
        
        prediction_an[:, j, i] = yy_pred[:,2]
        prediction_nn[:, j, i] = yy_pred[:,1]
        prediction_bn[:, j, i] = yy_pred[:,0]
        
        testing[:, j, i] = y_test
        
        training_acc[0, j, i] = training_accuracy 
        testing_acc[0, j, i] = accuracy   
        
        j = j + 1 
    
        print(1)
        
    i = i + 1

# Make an xarray of predictions data

time_step = [i for i in range(0, len(yy_pred))]

longitude = longitude.values
latitude = latitude.values

predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),      
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions_an",
    ),
)

predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),      
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions_nn",
    ),
)

predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),      
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions_bn",
    ),
)

# Make an xarray of tests data

test = xr.DataArray(
    data=testing,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),       
        time_step= (["time_step"], time_step)
    ),
    attrs=dict(
        description="predictions",
    ),
)

# make categories from the predictions

categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])

categorical_p = categories.rename({'time_step':'forecast_time'})

# Make an xarray for the testing accuracies

test_acc = xr.DataArray(
    data=testing_acc,
    dims=["training_accuracy", "latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),       
    ),
    attrs=dict(
        description="testing accuracy",
    ),
)

# Make an xarray for the training accuracies

train_acc = xr.DataArray(
    data=training_acc,
    dims=["training_accuracy", "latitude", "longitude"],
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),       
    ),
    attrs=dict(
        description="testing accuracy",
    ),
)

## RPSS

In [ ]:
import xskillscore as xs
from scripts import  make_probabilistic

In [ ]:
hindcast_ecmwfter = make_probabilistic(hindcast_ecmwf, tercile_edges = tercile_edges)
hindcast_observprob = make_probabilistic(hindcast_observation, tercile_edges = tercile_edges)

In [ ]:
del hindcast_ecmwf['week']
del hindcast_ecmwf['year']

In [ ]:
obs_p = hindcast_observprob['tp'].isel(lead_time = 0).sel(forecast_time = slice('2018-01-01', '2019-12-30'))

In [ ]:
clim_p = xr.DataArray([1/3, 1/3, 1/3], dims='category', coords={'category':['below normal', 'near normal', 'above normal']}).to_dataset(name='tp')
clim_p['t2m'] = clim_p['tp']

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
# region1 = rpss_cl.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
# region2 = rpss_cl.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
# region3 = rpss_cl.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

def RPSS(region, categorical, start, end):
        
    # regions

    region1 = hindcast_observation.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hindcast_observation.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hindcast_observation.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        long = region1.longitude
        lat  = region1.latitude
        
    if region == 'region2':
        long = region2.longitude
        lat  = region2.latitude
    
    if region == 'region3':
        long = region3.longitude
        lat  = region3.latitude
    
    obs_p = hindcast_observprob['tp'].isel(lead_time = 0).sel(forecast_time = slice(start, end)).sel(longitude = long).sel(latitude = lat)
    ecmwf_p = hindcast_ecmwfter['tp'].isel(lead_time = 0).sel(forecast_time = slice(start, end)).sel(longitude = long).sel(latitude = lat)
    
    rps_cl = xs.rps(obs_p, clim_p['tp'], category_edges=None, dim=[], input_distributions='p').compute()
    rps_ecmwf = xs.rps(obs_p, ecmwf_p, category_edges=None, dim=[], input_distributions='p').compute()
    rps_model = xs.rps(obs_p, categorical, category_edges=None, dim=[], input_distributions='p').compute()
    
    rpss_cl = 1 - (rps_model.mean('forecast_time') / rps_cl.mean('forecast_time'))
    rpss_ecmwf = 1 - (rps_model.mean('forecast_time') / rps_ecmwf.mean('forecast_time'))  
    
    RPSS_cl = rpss_cl.mean(dim = 'latitude').mean(dim = 'longitude').values
    RPSS_ecmwf = rpss_ecmwf.mean(dim = 'latitude').mean(dim = 'longitude').values
    
    print(RPSS_cl, RPSS_ecmwf)
    
    return RPSS_cl, RPSS_ecmwf

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS('region3', categorical_p)

## ROC curve

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
def ROC_ML(dataset, probabilistic, tercile_edges, longitude, latitude, variable, lead_time, start, end):
    aprob = probabilistic.sel(forecast_time= slice(start, end)).sel(longitude = longitude, latitude = latitude, method = 'nearest').isel(category = 2).values
    nnprob = probabilistic.sel(forecast_time= slice(start, end)).sel(longitude = longitude, latitude = latitude, method = 'nearest').isel(category = 1).values
    bprob = probabilistic.sel(forecast_time= slice(start, end)).sel(longitude = longitude, latitude = latitude, method = 'nearest').isel(category = 0).values
    
    x_plus = dataset[variable].sel(forecast_time= slice(start, end)).isel(lead_time = lead_time).sel(longitude = longitude, latitude = latitude, method = 'nearest')

    upper = []
    lower = []
    for i in range(0,len(dataset.forecast_time)):
        if i < 53:
            u = tercile_edges[variable].isel(week = i, category_edge = 1, lead_time = lead_time).sel(longitude = longitude, latitude = latitude, method = 'nearest').values
            upper.append(u)
            l = tercile_edges[variable].isel(week = i, category_edge = 0, lead_time = lead_time).sel(longitude = longitude, latitude = latitude, method = 'nearest').values
            lower.append(l)

    for i in range(0,(int(len(dataset.forecast_time)/53)-1)):
        for j in range(0, 53):
            upper.append(upper[j])
            lower.append(lower[j])
            
    dataset['upper']=(['forecast_time'], upper)
    dataset['lower']=(['forecast_time'], lower)

    u = dataset.upper.sel(forecast_time= slice(start, end))

    l = dataset.lower.sel(forecast_time= slice(start, end))

    
    forecast_time = x_plus.forecast_time
    
    above = []
    nnormal = []
    below = []
    
    
    for i in range(0, len(forecast_time)):
        
        if x_plus.isel(forecast_time = i).values > u.isel(forecast_time = i):
            a = 1
            above.append(a)
        else:
            a = 0
            above.append(a)
            
        if l.isel(forecast_time = i) < x_plus.isel(forecast_time =  i) < u.isel(forecast_time = i):
            nn = 1
            nnormal.append(nn)
        else:
            nn = 0
            nnormal.append(nn)
        
        if x_plus.isel(forecast_time = i) < l.isel(forecast_time = i):

            b = 1
            below.append(b)
        else:
            b = 0
            below.append(b)
    
    
    ns_probs = [0 for _ in range(len(aprob))]
    # calculate scores
    ns_auc = roc_auc_score(above, ns_probs)
    nma_auc = roc_auc_score(above, aprob)
    nmnn_auc = roc_auc_score(nnormal, nnprob)
    nmb_auc = roc_auc_score(below, bprob)
    
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(above, ns_probs)
    nma_fpr, nma_tpr, _ = roc_curve(above, aprob)
    nmnn_fpr, nmnn_tpr, _ = roc_curve(nnormal, nnprob)
    nmb_fpr, nmb_tpr, _ = roc_curve(below, bprob)
    
    # plot the ROC
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
    axes.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    axes.plot(nma_fpr, nma_tpr, color = 'r', label = 'Above normal, AUC = {:.3f}'.format(nma_auc))
    axes.plot(nmnn_fpr, nmnn_tpr, color = 'g', label = 'Near normal, AUC = {:.3f}'.format(nmnn_auc))
    axes.plot(nmb_fpr, nmb_tpr, color = 'y', label = 'Below normal, AUC = {:.3f}'.format(nmb_auc))
    axes.set_ylim(ymin = 0)
    axes.set_xlim(xmin = 0)
    axes.set(xlabel= 'False Positive Rate', ylabel='True Positive Rate', title = 'Receiver operating characteristic curve (ROC)')
    axes.legend(loc="upper left")
    
    del dataset['upper']
    del dataset['lower']
    
    return aprob, nnprob, bprob, above, nnormal, below

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
# region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
# NNl_roc3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

## Logistic regression (LR), Multilayer perceptron (MLP) and random forest (RF)

In [ ]:
# the random forest model is added to the logistic regression and the multi-layer perceptron model

def ML_models(model, region):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # claculate the classes
    obs00_19_terTR = obs00_19_ter.assign_coords(category=[0, 1, 2])
    classes = (obs00_19_terTR * obs00_19_terTR.category).sum('category')
    classes = classes.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))

    
    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude
        
    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude
    
    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results

    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

    prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    
    testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    
    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            # training data

            x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2000-01-01', '2017-12-30'))
            xx = x.to_dataframe().reset_index()
            xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar = xx_inp[['tp','t2m']]
            x_train = xx_inpVar.values

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2017-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]
            y_train = yy_inpVar.values.ravel()

            # testing data

            x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))
            xx = x.to_dataframe().reset_index()
            xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar = xx_inp[['tp','t2m']]
            x_test = xx_inpVar.values

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2018-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]
            y_test = yy_inpVar.values.ravel()


            # Training the LR and MLP models
            
            min_max_scaler = preprocessing.MinMaxScaler()
            x_trainT = min_max_scaler.fit_transform(x_train)
            x_testT = min_max_scaler.fit_transform(x_test)

            if  model == 'LR':
                lr = LogisticRegression()
                lr_model = lr.fit(x_trainT, y_train)
                training_accuracy = lr_model.score(x_train, y_train)
#                 print(training_accuracy)

            if model == 'RF':
                
                rf = RandomForestClassifier()
                rf_model = rf.fit(x_trainT, y_train)
                training_accuracy = rf_model.score(x_train, y_train)

            if model == 'MLP':
        
                # define model
                mlp_model = Sequential()
                mlp_model.add(Dense(500, input_dim=2, activation='relu'))
                mlp_model.add(Dense(3, activation='softmax'))
                mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


                # simple early stopping
                es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
                mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

                # fit model
                history = mlp_model.fit(x_trainT, y_train, validation_data=(x_testT, y_test), epochs=4000, verbose=0, callbacks=[es, mc])

                # load the saved model
                saved_model = load_model('best_model.h5')

                # evaluate the model
                _, train_acc = saved_model.evaluate(x_trainT, y_train, verbose=0)
                _, test_acc = saved_model.evaluate(x_testT, y_test, verbose=0)
                training_accuracy = train_acc
                accuracy = test_acc

            # Testing the LR and MLP models
            
            if  model == 'LR':
                yy_pred = lr_model.predict_proba(x_testT)
                accuracy = lr_model.score(x_testT, y_test)
                
            if model == 'RF':
                yy_pred = rf_model.predict_proba(x_testT)
                accuracy = rf_model.score(x_testT, y_test) 
                
            if model == 'MLP':
                yy_pred = saved_model.predict(x_testT)

                
            # add the values to the empty array
            
            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            testing[:, j, i] = y_test
            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy          
        
            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data
    
    time_step = [i for i in range(0, len(yy_pred))]
    
    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
        data=prediction_an,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_an",
        ),
    )

    predict_nn = xr.DataArray(
        data=prediction_nn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_nn",
        ),
    )

    predict_bn = xr.DataArray(
        data=prediction_bn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_bn",
        ),
    )
    
    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})
    

    # Make an xarray of tests data

    test = xr.DataArray(
        data=testing,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions",
        ),
    )
    
    # Make an xarray for the testing accuracies
    
    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    # Make an xarray for the training accuracies
    
    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    return test, test_acc, train_acc, categorical_p

In [ ]:
test, test_acc, train_acc, categorical_p = ML_models('RF', 'region3')

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS('region3', categorical_p, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
# region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
NNl_roc3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

In [ ]:
# The training and testing data are added using one loop and the iloc command is used to select the training and testing datasets
# the random forest model is added to the logistic regression and the multi-layer perceptron model

def ML_models_cor(model, region, spatial):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # claculate the classes
    obs00_19_terTR = obs00_19_ter.assign_coords(category=[0, 1, 2])
    classes = (obs00_19_terTR * obs00_19_terTR.category).sum('category')
    classes = classes.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))

    
    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude
        
    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude
    
    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results

    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-02', '2019-12-24'))

    prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    
    testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    
    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            # training data

            x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            xx = x.to_dataframe().reset_index()
            
            # add the correlated inputs
            xx['tp_lag1'] = xx.tp.shift(1).bfill()
            
            if spatial == 'one':
                disla = lat - (1 * 1.5)
                dislo = long - (1 * 1.5)
                dla = np.arange(0, 4.5, 1.5)
                dlo = np.arange(0, 4.5, 1.5)
                v = 'var 4'
                
            if spatial == 'two':
                disla = lat - (2 * 1.5)
                dislo = long - (2 * 1.5)
                dla = np.arange(0, 7.5, 1.5)
                dlo = np.arange(0, 7.5, 1.5)
                v = 'var 12'
                
            if spatial == 'three':
                disla = lat - (3 * 1.5)
                dislo = long - (3 * 1.5)
                dla = np.arange(0, 10.5, 1.5)
                dlo = np.arange(0, 10.5, 1.5)
                v = 'var 24'
                
            # var = []
            nam = 0
            for ii in dlo:
                for jj in dla:

                    xx['Var'] = hecm.tp.sel(latitude = disla + jj, longitude = dislo + ii).sel(forecast_time = slice('2000-01-01', '2019-12-30')).mean(dim = 'realization')
                    name = 'var {}'.format(nam)
                    xx.rename(columns={'Var': name}, inplace=True)
                    nam = nam + 1
                    
            
            xx_dropna = xx.dropna(how='all', axis=1)
            xx_inp_train = xx_dropna.loc[0:953]
            xx_inp_tr = xx_inp_train.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar_train = xx_inp_tr.drop([v], axis = 'columns')
            
            x_train = xx_inpVar_train.values

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]
            
            y_train = yy_inpVar.loc[0:953].values.ravel()

            # testing data
            xx_inp_test = xx_dropna.loc[954:]
            xx_inp_te = xx_inp_test.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar_test = xx_inp_te.drop([v], axis = 'columns')
            
            x_test = xx_inpVar_test.values

            y_test = yy_inpVar.loc[954:].values.ravel()

            # Training the LR and MLP models
            
            min_max_scaler = preprocessing.MinMaxScaler()
            x_trainT = min_max_scaler.fit_transform(x_train)
            x_testT = min_max_scaler.fit_transform(x_test)

            if  model == 'LR':
                lr = LogisticRegression()
                lr_model = lr.fit(x_trainT, y_train)
                training_accuracy = lr_model.score(x_train, y_train)
#                 print(training_accuracy)

            if model == 'RF':
                
                rf = RandomForestClassifier()
                rf_model = rf.fit(x_trainT, y_train)
                training_accuracy = rf_model.score(x_train, y_train)

            if model == 'MLP':
        
                # define model
                mlp_model = Sequential()
                mlp_model.add(Dense(500, input_dim=len(xx_inpVar_train.columns), activation='relu'))
                mlp_model.add(Dense(3, activation='softmax'))
                mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


                # simple early stopping
                es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
                mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

                # fit model
                history = mlp_model.fit(x_trainT, y_train, validation_data=(x_testT, y_test), epochs=4000, verbose=0, callbacks=[es, mc])

                # load the saved model
                saved_model = load_model('best_model.h5')

                # evaluate the model
                _, train_acc = saved_model.evaluate(x_trainT, y_train, verbose=0)
                _, test_acc = saved_model.evaluate(x_testT, y_test, verbose=0)
                training_accuracy = train_acc
                accuracy = test_acc

            # Testing the LR and MLP models
            
            if  model == 'LR':
                yy_pred = lr_model.predict_proba(x_testT)
                accuracy = lr_model.score(x_testT, y_test)
                
            if model == 'RF':
                yy_pred = rf_model.predict_proba(x_testT)
                accuracy = rf_model.score(x_testT, y_test)
            
            if model == 'MLP':
                yy_pred = saved_model.predict(x_testT)

                
            # add the values to the empty array
            
            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            testing[:, j, i] = y_test
            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy          
        
            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data
    
    time_step = [i for i in range(0, len(yy_pred))]
    
    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
        data=prediction_an,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_an",
        ),
    )

    predict_nn = xr.DataArray(
        data=prediction_nn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_nn",
        ),
    )

    predict_bn = xr.DataArray(
        data=prediction_bn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_bn",
        ),
    )
    
    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})
    

    # Make an xarray of tests data

    test = xr.DataArray(
        data=testing,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions",
        ),
    )
    
    # Make an xarray for the testing accuracies
    
    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    # Make an xarray for the training accuracies
    
    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    return xx_inp_train, test, test_acc, train_acc, categorical_p

In [ ]:
xx_inp_train, test, test_acc, train_acc, categorical_p = ML_models_cor('RF', 'region3', 'two')

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS('region2', categorical_p, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
# NNl_roc3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

## LSTM Model

In [ ]:
import pandas as pd

from keras.preprocessing.sequence import TimeseriesGenerator
from numpy import hstack
from numpy import insert

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:

def ML_LSTM(region):
    lead_time = 0
    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # claculate the classes
    obs00_19_terTR = obs00_19_ter.assign_coords(category=[0, 1, 2])
    classes = (obs00_19_terTR * obs00_19_terTR.category).sum('category')
    classes = classes.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))

    
    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude
        
    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude
    
    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results

    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

    prediction_an = np.zeros((105, len(latitude), len(longitude)))
    prediction_nn = np.zeros((105, len(latitude), len(longitude)))
    prediction_bn = np.zeros((105, len(latitude), len(longitude)))
    
    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # training and testing data

            x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            xx = x.to_dataframe().reset_index()
            xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar = xx_inp[['tp','t2m']]

            y = target_cl.sel(longitude = long, latitude = lat, method = 'nearest').sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            x_train = xx_inpVar.loc[0:953]
            x_train_values = x_train.values

            x_test = xx_inpVar.loc[901:]
            x_test_values = x_test.values


            y_train = yy_inpVar.loc[0:953]
            y_train_values = y_train.values.ravel()

            y_test = yy_inpVar.loc[901:]
            y_test_values = y_test.values.ravel()

            # using the minimum and maximum scalar to normalize the data

            x_trainT = min_max_scaler.fit_transform(x_train_values)
            x_testT = min_max_scaler.fit_transform(x_test_values)

            # data preprocessing for the LSTM technique
            # for training

            ytrdata = insert(y_train_values, 0, 1000)
            print('the target ouptut shape for training')
            print(ytrdata.shape)

            xtrtp = insert(x_trainT[:,0], (954), 1000)
            xtrt2m = insert(x_trainT[:,1], (954), 1000)

            xtrtpre = xtrtp.reshape(len(xtrtp), 1)
            xtrt2mre = xtrt2m.reshape(len(xtrt2m), 1)

            xtrdata = hstack((xtrtpre, xtrt2mre))
            print('training input shape')
            print(xtrdata.shape)

            # for testing

            ytedata = insert(y_test_values, 0, 1000)
            print('the target ouptut shape for testing')
            print(ytedata.shape)

            xtetp = insert(x_testT[:,0], (158), 1000)
            xtet2m = insert(x_testT[:,1], (158), 1000)

            xtetpre = xtetp.reshape(len(xtetp), 1)
            xtet2mre = xtet2m.reshape(len(xtet2m), 1)

            xtedata = hstack((xtetpre, xtet2mre))
            print('testing input shape')
            print(xtedata.shape)

            # using the timeseries generator to generate the data
            n_input = 54
            trgenerator = TimeseriesGenerator(xtrdata, ytrdata, length=n_input, batch_size=1)
            tegenerator = TimeseriesGenerator(xtedata, ytedata, length=n_input, batch_size=1)

            # built the LSTM architecture

            # define model

            lstm_model = Sequential()
            lstm_model.add(LSTM(100, input_shape= (n_input, x_train_values.shape[1]), activation='relu'))
            lstm_model.add(Dense(3, activation='softmax'))
            lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            # simple early stopping
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
            mc = ModelCheckpoint('models/LSTM'+ region +'/model_' + str(lead_time) + '_' +  str(i) + str(j) + '.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

            # fit the model
            lstm_model.fit(trgenerator, validation_data= tegenerator, epochs=4000, verbose=0, callbacks=[es, mc])

            # load the model
            Model = load_model('models/LSTM' + region + '/model_' + str(lead_time) + '_' +  str(i) + str(j) + '.h5')

            # evaluate the model
            _, train_acc = Model.evaluate(trgenerator, verbose=0)
            _, test_acc = Model.evaluate(tegenerator, verbose=0)
            training_accuracy = train_acc
            accuracy = test_acc

            # make predictions
            yy_pred = np.zeros((len(tegenerator),3))

            for g in range(len(tegenerator)):
                x, y = tegenerator[g]
                ypredict = Model.predict(x)
                yy_pred[g, :] = ypredict
            
            # add the values to the empty array   
            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy          
        
            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data
    
    time_step = [i for i in range(0, len(yy_pred))]
    
    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
        data=prediction_an,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_an",
        ),
    )

    predict_nn = xr.DataArray(
        data=prediction_nn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_nn",
        ),
    )

    predict_bn = xr.DataArray(
        data=prediction_bn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_bn",
        ),
    )
    
    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})
    
    # Make an xarray for the testing accuracies
    
    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    # Make an xarray for the training accuracies
    
    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    return test_acc, train_acc, categorical_p

In [ ]:
# The training and testing data are added using one loop and the iloc command is used to select the training and testing datasets
# the random forest model is added to the logistic regression and the multi-layer perceptron model

def ML_LSTM_cor(region, spatial):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # claculate the classes
    obs00_19_terTR = obs00_19_ter.assign_coords(category=[0, 1, 2])
    classes = (obs00_19_terTR * obs00_19_terTR.category).sum('category')
    classes = classes.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))

    
    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude
        
    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude
    
    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results

    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-02', '2019-12-24'))

    prediction_an = np.zeros((105, len(latitude), len(longitude)))
    prediction_nn = np.zeros((105, len(latitude), len(longitude)))
    prediction_bn = np.zeros((105, len(latitude), len(longitude)))
    
    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            # training data

            x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            xx = x.to_dataframe().reset_index()
            
            # add the correlated inputs
            xx['tp_lag1'] = xx.tp.shift(1).bfill()
            
            if spatial == 'one':
                disla = lat - (1 * 1.5)
                dislo = long - (1 * 1.5)
                dla = np.arange(0, 4.5, 1.5)
                dlo = np.arange(0, 4.5, 1.5)
                v = 'var 4'
                
            if spatial == 'two':
                disla = lat - (2 * 1.5)
                dislo = long - (2 * 1.5)
                dla = np.arange(0, 7.5, 1.5)
                dlo = np.arange(0, 7.5, 1.5)
                v = 'var 12'
                
            if spatial == 'three':
                disla = lat - (3 * 1.5)
                dislo = long - (3 * 1.5)
                dla = np.arange(0, 10.5, 1.5)
                dlo = np.arange(0, 10.5, 1.5)
                v = 'var 24'
                
            # var = []
            nam = 0
            for ii in dlo:
                for jj in dla:

                    xx['Var'] = hecm.tp.sel(latitude = disla + jj, longitude = dislo + ii).sel(forecast_time = slice('2000-01-01', '2019-12-30')).mean(dim = 'realization')
                    name = 'var {}'.format(nam)
                    xx.rename(columns={'Var': name}, inplace=True)
                    nam = nam + 1
            
            xx_dropna = xx.dropna(how='all', axis=1)
            xx_inp_train = xx_dropna.loc[0:953]
            
            # adding one row to the end of the data
            new_data = pd.DataFrame(xx_inp_train[-1:].values,  index=[954], columns= xx_inp_train.columns)
            xx_inp_train = xx_inp_train.append(new_data)
            
            xx_inp_tr = xx_inp_train.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar_train = xx_inp_tr.drop([v], axis = 'columns')
            
            x_train = xx_inpVar_train.values

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]
            
            y_train = yy_inpVar.loc[0:953].values.ravel()

            # testing data
            xx_inp_test = xx_dropna.loc[901:]
            
            # adding one row to the end of the data
            new_data = pd.DataFrame(xx_inp_test[-1:].values,  index=[159], columns= xx_inp_test.columns)
            xx_inp_test = xx_inp_test.append(new_data)
            
            xx_inp_te = xx_inp_test.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
            xx_inpVar_test = xx_inp_te.drop([v], axis = 'columns')
            
            x_test = xx_inpVar_test.values

            y_test = yy_inpVar.loc[901:].values.ravel()

            # Training the LR and MLP models
            
            min_max_scaler = preprocessing.MinMaxScaler()
            x_trainT = min_max_scaler.fit_transform(x_train)
            x_testT = min_max_scaler.fit_transform(x_test)

            # data preprocessing for the LSTM technique
            # for training

            ytrdata = insert(y_train, 0, 1000)
            print('the target ouptut shape for training')
            print(ytrdata.shape)
            
            xtrdata = x_trainT
            print('training input shape')
            print(xtrdata.shape)

            # for testing

            ytedata = insert(y_test, 0, 1000)
            print('the target ouptut shape for testing')
            print(ytedata.shape)

            xtedata = x_testT
            print('testing input shape')
            print(xtedata.shape)

            # using the timeseries generator to generate the data
            n_input = 54
            trgenerator = TimeseriesGenerator(xtrdata, ytrdata, length=n_input, batch_size=1)
            tegenerator = TimeseriesGenerator(xtedata, ytedata, length=n_input, batch_size=1)

            # built the LSTM architecture

            # define model

            lstm_model = Sequential()
            lstm_model.add(LSTM(100, input_shape= (n_input, x_train_values.shape[1]), activation='relu'))
            lstm_model.add(Dense(3, activation='softmax'))
            lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

            # simple early stopping
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
            mc = ModelCheckpoint('models/LSTM'+ region +'/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)''

            # fit the model
            lstm_model.fit(trgenerator, validation_data= tegenerator, epochs=4000, verbose=0, callbacks=[es, mc])

            # load the model
            Model = load_model('models/LSTM' + region + '/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5')
            
            # evaluate the model
            _, train_acc = Model.evaluate(trgenerator, verbose=0)
            _, test_acc = Model.evaluate(tegenerator, verbose=0)
            training_accuracy = train_acc
            accuracy = test_acc

            # make predictions
            yy_pred = np.zeros((len(tegenerator),3))

            for g in range(len(tegenerator)):
                x, y = tegenerator[g]
                ypredict = Model.predict(x)
                yy_pred[g, :] = ypredict
            
            # add the values to the empty array
            
            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy          
        
            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data
    
    time_step = [i for i in range(0, len(yy_pred))]
    
    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
        data=prediction_an,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_an",
        ),
    )

    predict_nn = xr.DataArray(
        data=prediction_nn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_nn",
        ),
    )

    predict_bn = xr.DataArray(
        data=prediction_bn,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),      
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions_bn",
        ),
    )
    
    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})
    
    # Make an xarray for the testing accuracies
    
    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    # Make an xarray for the training accuracies
    
    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    
    return xx_inp_train, test_acc, train_acc, categorical_p

In [ ]:
xx_inp_train, test_acc, train_acc, categorical_p = ML_LSTM_cor('RF', 'region3', 'two')

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS('region2', categorical_p, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
# NNl_roc3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

## Committee model (CM)

In [ ]:
# Function of the stacked model
def CM(region, model, lead_time):


    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude

    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude

    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results
    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

    prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            stackX = None

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            yy_inpVar_train = yy_inpVar.loc[0:953]
            y_train = yy_inpVar_train.values.ravel()

            yy_inpVar_test = yy_inpVar.loc[954:]
            y_test = yy_inpVar_test.values.ravel()

            for r in range(0,11): 

                # training and testing data

                x = hecm_int.isel(realization = r).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
                xx = x.to_dataframe().reset_index()
                xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
                xx_inpVar = xx_inp[['tp','t2m']]

                xx_inpVar_train = xx_inpVar.loc[0:953]
                x_train = xx_inpVar_train.values

                xx_inpVar_test = xx_inpVar.loc[954:]
                x_test = xx_inpVar_test.values

                # Training the LR and MLP models

                min_max_scaler = preprocessing.MinMaxScaler()
                x_trainT = min_max_scaler.fit_transform(x_train)
                x_testT = min_max_scaler.fit_transform(x_test)

                if  model == 'LR':
                    lr = LogisticRegression()
                    lr_model = lr.fit(x_trainT, y_train)
                    training_accuracy = lr_model.score(x_train, y_train)
    #                 print(training_accuracy)

                if model == 'MLP':

                    # define model
                    mlp_model = Sequential()
                    mlp_model.add(Dense(500, input_dim=len(xx_inpVar_train.columns), activation='relu'))
                    mlp_model.add(Dense(3, activation='softmax'))
                    mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


                    # simple early stopping
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
                    mc = ModelCheckpoint('models/'+ region +'/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

                    # fit model
                    Model = mlp_model.fit(x_trainT, y_train, validation_data=(x_testT, y_test), epochs=4000, verbose=0, callbacks=[es, mc])        

                    # load the model
                    Model = load_model('models/' + region + '/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5')

                    # make prediction
                    yhat = Model.predict(x_testT, verbose=0)

                    # stack predictions into [rows, members, probabilities]
                    if stackX is None:
                        stackX = yhat
                    else:
                        stackX = dstack((stackX, yhat))

            stackXX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))

            meta_model = LogisticRegression()

            meta_model.fit(stackXX, y_test)

            training_accuracy = meta_model.score(stackXX, y_test)
            accuracy = training_accuracy

            yy_pred = meta_model.predict_proba(stackXX)

            # save the meta_model
            pickle.dump(meta_model, open('models/' + region + '/meta_model_' + str(lead_time) + '_' +  str(i) + str(j) + '.sav', 'wb'))

            # add the values to the empty array

            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            testing[:, j, i] = y_test
            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy   

            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data

    time_step = [i for i in range(0, len(yy_pred))]

    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_an",
    ),
    )

    predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_nn",
    ),
    )

    predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_bn",
    ),
    )

    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})

    # Make an xarray of tests data

    test = xr.DataArray(
        data=testing,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions",
        ),
    )

    # Make an xarray for the testing accuracies

    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )

    # Make an xarray for the training accuracies

    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    return test_acc, train_acc, categorical_p

In [ ]:
test_acc, train_acc, categorical_p = CM('region1', 'MLP', 0)

In [ ]:
obs_p = hindcast_observprob['tp'].isel(lead_time = 0).sel(forecast_time = slice('2018-01-01', '2019-12-30'))

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
# region1 = rpss_cl.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
# region2 = rpss_cl.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
# region3 = rpss_cl.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

def RPSS12(region, categorical, lead_time, start, end):
        
    # regions

    region1 = hindcast_observation.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hindcast_observation.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hindcast_observation.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        long = region1.longitude
        lat  = region1.latitude
        
    if region == 'region2':
        long = region2.longitude
        lat  = region2.latitude
    
    if region == 'region3':
        long = region3.longitude
        lat  = region3.latitude
    
    obs_p = hindcast_observprob['tp'].isel(lead_time = lead_time).sel(forecast_time = slice(start, end)).sel(longitude = long).sel(latitude = lat)
    ecmwf_p = hindcast_ecmwfter['tp'].isel(lead_time = lead_time).sel(forecast_time = slice(start, end)).sel(longitude = long).sel(latitude = lat)
    
    rps_cl = xs.rps(obs_p, clim_p['tp'], category_edges=None, dim=[], input_distributions='p').compute()
    rps_ecmwf = xs.rps(obs_p, ecmwf_p, category_edges=None, dim=[], input_distributions='p').compute()
    rps_model = xs.rps(obs_p, categorical, category_edges=None, dim=[], input_distributions='p').compute()
    
    rpss_cl = 1 - (rps_model.mean('forecast_time') / rps_cl.mean('forecast_time'))
    rpss_ecmwf = 1 - (rps_model.mean('forecast_time') / rps_ecmwf.mean('forecast_time'))  
    
    RPSS_cl = rpss_cl.mean(dim = 'latitude').mean(dim = 'longitude').values
    RPSS_ecmwf = rpss_ecmwf.mean(dim = 'latitude').mean(dim = 'longitude').values
    
    print(RPSS_cl, RPSS_ecmwf)
    
    return RPSS_cl, RPSS_ecmwf

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS12('region3', categorical_p, 0, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
# region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
NNl_roc3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

In [ ]:
# Function of the stacked model
# Adding the correlated inputs

def CMcor(region, spatial, model, lead_time):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_interpolate = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude

    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude

    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results
    x = hecm_interpolate.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

    prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            stackX = None

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # adding the target outputs for training and testing

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            yy_inpVar_train = yy_inpVar.loc[0:953]
            y_train = yy_inpVar_train.values.ravel()

            yy_inpVar_test = yy_inpVar.loc[954:]
            y_test = yy_inpVar_test.values.ravel()

            for r in range(0,11): 

                # add the correlated inputs
                x = hecm_int.isel(realization = r).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
                xx = x.to_dataframe().reset_index()
                xx['tp_lag1'] = xx.tp.shift(1).bfill()

                if spatial == 'one':
                    disla = lat - (1 * 1.5)
                    dislo = long - (1 * 1.5)
                    dla = np.arange(0, 4.5, 1.5)
                    dlo = np.arange(0, 4.5, 1.5)
                    v = 'var 4'

                if spatial == 'two':
                    disla = lat - (2 * 1.5)
                    dislo = long - (2 * 1.5)
                    dla = np.arange(0, 7.5, 1.5)
                    dlo = np.arange(0, 7.5, 1.5)
                    v = 'var 12'

                if spatial == 'three':
                    disla = lat - (3 * 1.5)
                    dislo = long - (3 * 1.5)
                    dla = np.arange(0, 10.5, 1.5)
                    dlo = np.arange(0, 10.5, 1.5)
                    v = 'var 24'

                # var = []
                nam = 0
                for ii in dlo:
                    for jj in dla:

                        xx['Var'] = hecm_interpolate.tp.sel(latitude = disla + jj, longitude = dislo + ii).sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(realization = r)
                        name = 'var {}'.format(nam)
                        xx.rename(columns={'Var': name}, inplace=True)
                        nam = nam + 1


                xx_dropna = xx.dropna(how='all', axis=1)
                xx_inp_train = xx_dropna.loc[0:953]
                xx_inp_tr = xx_inp_train.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time', 'realization'], axis = 'columns')
                xx_inpVar_train = xx_inp_tr.drop([v], axis = 'columns')

                x_train = xx_inpVar_train.values
                print(x_train.shape)

                # testing data
                xx_inp_test = xx_dropna.loc[954:]
                xx_inp_te = xx_inp_test.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time', 'realization'], axis = 'columns')
                xx_inpVar_test = xx_inp_te.drop([v], axis = 'columns')

                x_test = xx_inpVar_test.values

                # Training the LR and MLP models

                min_max_scaler = preprocessing.MinMaxScaler()
                x_trainT = min_max_scaler.fit_transform(x_train)
                x_testT = min_max_scaler.fit_transform(x_test)

                if  model == 'LR':
                    lr = LogisticRegression()
                    lr_model = lr.fit(x_trainT, y_train)
                    training_accuracy = lr_model.score(x_train, y_train)
    #                 print(training_accuracy)

                if model == 'MLP':

                    # define model
                    mlp_model = Sequential()
                    mlp_model.add(Dense(500, input_dim=len(xx_inpVar_train.columns), activation='relu'))
                    mlp_model.add(Dense(3, activation='softmax'))
                    mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


                    # simple early stopping
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
                    mc = ModelCheckpoint('models/'+ region + spatial + '/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

                    # fit model
                    Model = mlp_model.fit(x_trainT, y_train, validation_data=(x_testT, y_test), epochs=4000, verbose=0, callbacks=[es, mc])        

                    # load the model
                    Model = load_model('models/' + region + spatial + '/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5')

                    # make prediction
                    yhat = Model.predict(x_testT, verbose=0)

                    # stack predictions into [rows, members, probabilities]
                    if stackX is None:
                        stackX = yhat
                    else:
                        stackX = dstack((stackX, yhat))

            stackXX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))

            stackXX = np.nan_to_num(stackXX)

            meta_model = LogisticRegression()

            meta_model.fit(stackXX, y_test)

            training_accuracy = meta_model.score(stackXX, y_test)
            accuracy = training_accuracy

            yy_pred = meta_model.predict_proba(stackXX)

            # save the meta_model
            pickle.dump(meta_model, open('models/' + region + spatial + '/meta_model_' + str(lead_time) + '_' +  str(i) + str(j) + '.sav', 'wb'))

            # add the values to the empty array

            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            testing[:, j, i] = y_test
            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy   

            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data

    time_step = [i for i in range(0, len(yy_pred))]

    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_an",
    ),
    )

    predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_nn",
    ),
    )

    predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_bn",
    ),
    )

    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})

    # Make an xarray of tests data

    test = xr.DataArray(
        data=testing,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions",
        ),
    )

    # Make an xarray for the testing accuracies

    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )

    # Make an xarray for the training accuracies

    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    return test_acc, train_acc, categorical_p

In [ ]:
test_acc, train_acc, categorical_p = CMcor('region1', 'one' , 'MLP', 0)

In [ ]:
obs_p = hindcast_observprob['tp'].isel(lead_time = 0).sel(forecast_time = slice('2018-01-01', '2019-12-30'))

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS12('region3', categorical_p, 0, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
# region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
NNl_roc3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

### In this case, the same procedure from above is used, the only difference is that, two of the last three years are used for CV and last year is used for testing

In [ ]:
# Function of the stacked model
# Using cross-validation and testing data

def CM2cv1te(region, model):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude

    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude

    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results
    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2019-01-01', '2019-12-30'))

    prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            stackX = None

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            yy_inpVar_train = yy_inpVar.loc[0:900]
            y_train = yy_inpVar_train.values.ravel()

            yy_inpVar_test = yy_inpVar.loc[901:]
            y_test = yy_inpVar_test.values.ravel()

            for r in range(0,11): 

                # training and testing data

                x = hecm_int.isel(realization = r).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
                xx = x.to_dataframe().reset_index()
                xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
                xx_inpVar = xx_inp[['tp','t2m']]

                xx_inpVar_train = xx_inpVar.loc[0:900]
                x_train = xx_inpVar_train.values

                xx_inpVar_test = xx_inpVar.loc[901:]
                x_test = xx_inpVar_test.values

                # Training the LR and MLP models

                min_max_scaler = preprocessing.MinMaxScaler()
                x_trainT = min_max_scaler.fit_transform(x_train)
                x_testT = min_max_scaler.fit_transform(x_test)

                if  model == 'LR':
                    lr = LogisticRegression()
                    lr_model = lr.fit(x_trainT, y_train)
                    training_accuracy = lr_model.score(x_train, y_train)
    #                 print(training_accuracy)

                if model == 'MLP':

                    # define model
                    mlp_model = Sequential()
                    mlp_model.add(Dense(500, input_dim=len(xx_inpVar_train.columns), activation='relu'))
                    mlp_model.add(Dense(3, activation='softmax'))
                    mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


                    # simple early stopping
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
                    mc = ModelCheckpoint('models/'+ region +'_ct/model_' + str(r + 1) + '_' +  str(i) + str(j) + '.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

                    # fit model
                    Model = mlp_model.fit(x_trainT, y_train, validation_data=(x_testT[0:106], y_test[0:106]), epochs=4000, verbose=0, callbacks=[es, mc])        

                    # load the model
                    Model = load_model('models/' + region + '_ct/model_' + str(r + 1) + '_' +  str(i) + str(j) + '.h5')

                    # make prediction
                    yhat = Model.predict(x_testT, verbose=0)

                    # stack predictions into [rows, members, probabilities]
                    if stackX is None:
                        stackX = yhat
                    else:
                        stackX = dstack((stackX, yhat))

            stackXX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))

            meta_model = LogisticRegression()

            meta_model.fit(stackXX[0:106], y_test[0:106])

            training_accuracy = meta_model.score(stackXX[0:106], y_test[0:106])

            yy_pred = meta_model.predict_proba(stackXX[106:])

            accuarcy = meta_model.score(stackXX[106:], y_test[106:])

            # save the meta_model
            import pickle
            pickle.dump(meta_model, open('models/' + region + '_ct/meta_model.sav', 'wb'))

            # add the values to the empty array

            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            testing[:, j, i] = y_test[106:]
            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy   

            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data

    time_step = [i for i in range(0, len(yy_pred))]

    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_an",
    ),
    )

    predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_nn",
    ),
    )

    predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_bn",
    ),
    )

    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})

    # Make an xarray of tests data

    test = xr.DataArray(
        data=testing,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions",
        ),
    )

    # Make an xarray for the testing accuracies

    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )

    # Make an xarray for the training accuracies

    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    return test_acc, train_acc, categorical_p

In [ ]:
test_acc, train_acc, categorical_p = CMcor('region1', 'one' , 'MLP', 0)

In [ ]:
obs_p = hindcast_observprob['tp'].isel(lead_time = 0).sel(forecast_time = slice('2018-01-01', '2019-12-30'))

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS12('region3', categorical_p, 0, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
# region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
region3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

In [ ]:
# Function of the stacked model
# Adding the correlated inputs

def CM2cv1tecor(region, spatial, model, lead_time):
    
    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_interpolate = hecm.interpolate_na(dim = 'forecast_time')

    target_cl = target_class.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).fillna(10).chunk(dict(forecast_time=-1))

    # hecm_int['cl'] = target_cl 
    # hobs_int['cl'] = target_cl

    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude

    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude

    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results
    x = hecm_interpolate.mean(dim = 'realization').sel(forecast_time = slice('2019-01-01', '2019-12-30'))

    prediction_an = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_nn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))
    prediction_bn = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    testing = np.zeros((len(x.forecast_time), len(latitude), len(longitude)))

    training_acc = np.zeros((1, len(latitude), len(longitude)))
    testing_acc = np.zeros((1, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            stackX = None

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # adding the target outputs for training and testing

            y = target_cl.sel(longitude = long, latitude = lat).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            yy_inpVar_train = yy_inpVar.loc[0:900]
            y_train = yy_inpVar_train.values.ravel()

            yy_inpVar_test = yy_inpVar.loc[901:]
            y_test = yy_inpVar_test.values.ravel()

            for r in range(0,11): 

                # add the correlated inputs
                x = hecm_int.isel(realization = r).sel(forecast_time = slice('2000-01-01', '2019-12-30'))
                xx = x.to_dataframe().reset_index()
                xx['tp_lag1'] = xx.tp.shift(1).bfill()

                if spatial == 'one':
                    disla = lat - (1 * 1.5)
                    dislo = long - (1 * 1.5)
                    dla = np.arange(0, 4.5, 1.5)
                    dlo = np.arange(0, 4.5, 1.5)
                    v = 'var 4'

                if spatial == 'two':
                    disla = lat - (2 * 1.5)
                    dislo = long - (2 * 1.5)
                    dla = np.arange(0, 7.5, 1.5)
                    dlo = np.arange(0, 7.5, 1.5)
                    v = 'var 12'

                if spatial == 'three':
                    disla = lat - (3 * 1.5)
                    dislo = long - (3 * 1.5)
                    dla = np.arange(0, 10.5, 1.5)
                    dlo = np.arange(0, 10.5, 1.5)
                    v = 'var 24'

                # var = []
                nam = 0
                for ii in dlo:
                    for jj in dla:

                        xx['Var'] = hecm_interpolate.tp.sel(latitude = disla + jj, longitude = dislo + ii).sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(realization = r)
                        name = 'var {}'.format(nam)
                        xx.rename(columns={'Var': name}, inplace=True)
                        nam = nam + 1


                xx_dropna = xx.dropna(how='all', axis=1)
                xx_inp_train = xx_dropna.loc[0:900]
                xx_inp_tr = xx_inp_train.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time', 'realization'], axis = 'columns')
                xx_inpVar_train = xx_inp_tr.drop([v], axis = 'columns')

                x_train = xx_inpVar_train.values
                print(x_train.shape)

                # testing data
                xx_inp_test = xx_dropna.loc[901:]
                xx_inp_te = xx_inp_test.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time', 'realization'], axis = 'columns')
                xx_inpVar_test = xx_inp_te.drop([v], axis = 'columns')

                x_test = xx_inpVar_test.values

                # Training the LR and MLP models

                min_max_scaler = preprocessing.MinMaxScaler()
                x_trainT = min_max_scaler.fit_transform(x_train)
                x_testT = min_max_scaler.fit_transform(x_test)

                if  model == 'LR':
                    lr = LogisticRegression()
                    lr_model = lr.fit(x_trainT, y_train)
                    training_accuracy = lr_model.score(x_train, y_train)
    #                 print(training_accuracy)

                if model == 'MLP':

                    # define model
                    mlp_model = Sequential()
                    mlp_model.add(Dense(500, input_dim=len(xx_inpVar_train.columns), activation='relu'))
                    mlp_model.add(Dense(3, activation='softmax'))
                    mlp_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


                    # simple early stopping
                    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
                    mc = ModelCheckpoint('models/'+ region + spatial + '_ct/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

                    # fit model
                    Model = mlp_model.fit(x_trainT, y_train, validation_data=(x_testT[0:106], y_test), epochs=4000, verbose=0, callbacks=[es, mc])        

                    # load the model
                    Model = load_model('models/' + region + spatial + '_ct/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5')

                    # make prediction
                    yhat = Model.predict(x_testT, verbose=0)

                    # stack predictions into [rows, members, probabilities]
                    if stackX is None:
                        stackX = yhat
                    else:
                        stackX = dstack((stackX, yhat))

            stackXX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))

            stackXX = np.nan_to_num(stackXX)

            meta_model = LogisticRegression()

            meta_model.fit(stackXX[0:106], y_test[0:106])

            training_accuracy = meta_model.score(stackXX[0:106], y_test[0:106])
            accuracy = training_accuracy

            yy_pred = meta_model.predict_proba(stackXX[106:])

            # save the meta_model
            pickle.dump(meta_model, open('models/' + region + spatial + '_ct/meta_model_' + str(lead_time) + '_' +  str(i) + str(j) + '.sav', 'wb'))

            # add the values to the empty array

            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            testing[:, j, i] = y_test[106:]
            training_acc[0, j, i] = training_accuracy 
            testing_acc[0, j, i] = accuracy   

            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data

    time_step = [i for i in range(0, len(yy_pred))]

    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_an",
    ),
    )

    predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_nn",
    ),
    )

    predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_bn",
    ),
    )

    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})

    # Make an xarray of tests data

    test = xr.DataArray(
        data=testing,
        dims=["time_step","latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
            time_step= (["time_step"], time_step)
        ),
        attrs=dict(
            description="predictions",
        ),
    )

    # Make an xarray for the testing accuracies

    test_acc = xr.DataArray(
        data=testing_acc,
        dims=["testing_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )

    # Make an xarray for the training accuracies

    train_acc = xr.DataArray(
        data=training_acc,
        dims=["training_accuracy", "latitude", "longitude"],
        coords=dict(
            longitude=(["longitude"], longitude),
            latitude=(["latitude"], latitude),       
        ),
        attrs=dict(
            description="testing accuracy",
        ),
    )
    return test_acc, train_acc, categorical_p

In [ ]:
test_acc, train_acc, categorical_p = CM2cv1tecor('region1', 'one' , 'MLP', 0)

In [ ]:
obs_p = hindcast_observprob['tp'].isel(lead_time = 0).sel(forecast_time = slice('2018-01-01', '2019-12-30'))

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl, RPSS_ecmwf = RPSS12('region3', categorical_p, 0, '2018-01-01', '2019-12-30')

In [ ]:
# region1 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2018-01-01', '2019-12-30')
# region2 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2018-01-01', '2019-12-30')
region3 = ROC_ML(hindcast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2018-01-01', '2019-12-30')

## Testing the CM using the real-time forecasting of 2020

In [ ]:
# create the classes from the CPC observation 

forecast_terTR = ter_forecast.assign_coords(category=[0, 1, 2])
classes = (forecast_terTR * forecast_terTR.category).sum('category')
class_forecast = classes.chunk(dict(forecast_time=-1))

In [ ]:
# Loading the saved models.
# Using the real-time forecasting of 2020 as input.

def CMte2020(region, model, lead_time):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    fecm = forecast_ecmwf.isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')
    fecm_interpolate = fecm.interpolate_na(dim = 'forecast_time')


    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude

    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude

    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results
    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

    prediction_an = np.zeros((53, len(latitude), len(longitude)))
    prediction_nn = np.zeros((53, len(latitude), len(longitude)))
    prediction_bn = np.zeros((53, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            stackX = None

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')

            fecm_cell = fecm.sel(longitude = long, latitude = lat)
            fecm_int = fecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            y = class_forecast.isel(lead_time = 0).sel(longitude = long, latitude = lat)
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            y_test = yy_inpVar.values.ravel()

            m = 1
            for r in range(0,11): 

                # training and testing data
                x = fecm_int.isel(realization = m)
                xx = x.to_dataframe().reset_index()

                xx_inp = xx.drop(['latitude','longitude', 'lead_time', 'valid_time'], axis = 'columns')
                xx_inpVar = xx_inp[['tp','t2m']]

                x_test = xx_inpVar.values

                # Training the LR and MLP models

                min_max_scaler = preprocessing.MinMaxScaler()
                x_testT = min_max_scaler.fit_transform(x_test)

                m = m + 1
    #             if r == 9:
    #                 m = 50

                if  model == 'LR':
                    lr = LogisticRegression()
                    lr_model = lr.fit(x_trainT, y_train)
                    training_accuracy = lr_model.score(x_train, y_train)
    #                 print(training_accuracy)

                if model == 'MLP':

                    # load the model
                    Model = load_model('models/'+ region +'/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5')

                    # make prediction
                    yhat = Model.predict(x_testT, verbose=0)

                    # stack predictions into [rows, members, probabilities]
                    if stackX is None:
                        stackX = yhat
                    else:
                        stackX = dstack((stackX, yhat))

            stackXX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
            stackXX = np.nan_to_num(stackXX)
            # Load the meta model

            meta_model = pickle.load(open('models/' + region + '/meta_model_' + str(lead_time) + '_' +  str(i) + str(j) + '.sav', 'rb'))

            yy_pred = meta_model.predict_proba(stackXX)

            # add the values to the empty array

            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data

    time_step = [i for i in range(0, len(yy_pred))]

    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_an",
    ),
    )

    predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_nn",
    ),
    )

    predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_bn",
    ),
    )

    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})
    
    return categorical_p

In [ ]:
categorical_p = CMte2020('region1','MLP', 0)

## RPSS and ROC curve for the  testing of the 2020 real-time forecasting

In [ ]:
import xskillscore as xs
from scripts import  make_probabilistic

In [ ]:
forecast_observprob = make_probabilistic(forecast_observation, tercile_edges = tercile_edges)

In [ ]:
del forecast_observation ['week']
del forecast_observation ['year']

In [ ]:
# forecast_observprob.to_netcdf('D:/Downloads/forecast_observprob.nc')

In [ ]:
obs_p = forecast_observprob['tp'].isel(lead_time = 0)

In [ ]:
clim_p = xr.DataArray([1/3, 1/3, 1/3], dims='category', coords={'category':['below normal', 'near normal', 'above normal']}).to_dataset(name='tp')
clim_p['t2m'] = clim_p['tp']

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
# region1 = rpss_cl.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
# region2 = rpss_cl.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
# region3 = rpss_cl.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

def RPSS_2020(region, categorical, lead_time, start, end):
        
    # regions

    region1 = hindcast_observation.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hindcast_observation.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hindcast_observation.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        long = region1.longitude
        lat  = region1.latitude
        
    if region == 'region2':
        long = region2.longitude
        lat  = region2.latitude
    
    if region == 'region3':
        long = region3.longitude
        lat  = region3.latitude
    
    obs_p = forecast_observprob['tp'].isel(lead_time = lead_time).sel(forecast_time = slice(start, end)).sel(longitude = long).sel(latitude = lat)
    
    rps_cl = xs.rps(obs_p, clim_p['tp'], category_edges=None, dim=[], input_distributions='p').compute()
    rps_model = xs.rps(obs_p, categorical, category_edges=None, dim=[], input_distributions='p').compute()
    
    rpss_cl = 1 - (rps_model.mean('forecast_time') / rps_cl.mean('forecast_time'))
    
    RPSS_cl = rpss_cl.mean(dim = 'latitude').mean(dim = 'longitude').values

    print(RPSS_cl)
    
    return RPSS_cl

In [ ]:
RPSS_cl = RPSS_2020('region3', categorical_p, 0, '2020-01-02', '2020-12-31')

In [ ]:
def ROC_ML2020(dataset, probabilistic, tercile_edges, longitude, latitude, variable, lead_time, start, end):
    aprob = probabilistic.sel(forecast_time= slice(start, end)).sel(longitude = longitude, latitude = latitude, method = 'nearest').isel(category = 2).values
    nnprob = probabilistic.sel(forecast_time= slice(start, end)).sel(longitude = longitude, latitude = latitude, method = 'nearest').isel(category = 1).values
    bprob = probabilistic.sel(forecast_time= slice(start, end)).sel(longitude = longitude, latitude = latitude, method = 'nearest').isel(category = 0).values
    
    x_plus = dataset[variable].sel(forecast_time= slice(start, end)).isel(lead_time = lead_time).sel(longitude = longitude, latitude = latitude, method = 'nearest')

    upper = []
    lower = []
    for i in range(0,len(dataset.forecast_time)):
        if i < 53:
            u = tercile_edges[variable].isel(week = i, category_edge = 1, lead_time = lead_time).sel(longitude = longitude, latitude = latitude, method = 'nearest').values
            upper.append(u)
            l = tercile_edges[variable].isel(week = i, category_edge = 0, lead_time = lead_time).sel(longitude = longitude, latitude = latitude, method = 'nearest').values
            lower.append(l)

    for i in range(0,(int(len(dataset.forecast_time)/53)-1)):
        for j in range(0, 53):
            upper.append(upper[j])
            lower.append(lower[j])
            
    dataset['upper']=(['forecast_time'], upper)
    dataset['lower']=(['forecast_time'], lower)

    u = dataset.upper.sel(forecast_time= slice(start, end))

    l = dataset.lower.sel(forecast_time= slice(start, end))

    
    forecast_time = x_plus.forecast_time
    
    above = []
    nnormal = []
    below = []
    
    
    for i in range(0, len(forecast_time)):
        
        if x_plus.isel(forecast_time = i).values > u.isel(forecast_time = i):
            a = 1
            above.append(a)
        else:
            a = 0
            above.append(a)
            
        if l.isel(forecast_time = i) < x_plus.isel(forecast_time =  i) < u.isel(forecast_time = i):
            nn = 1
            nnormal.append(nn)
        else:
            nn = 0
            nnormal.append(nn)
        
        if x_plus.isel(forecast_time = i) < l.isel(forecast_time = i):

            b = 1
            below.append(b)
        else:
            b = 0
            below.append(b)
    
    
    ns_probs = [0 for _ in range(len(aprob))]
    # calculate scores
    ns_auc = roc_auc_score(above, ns_probs)
    nma_auc = roc_auc_score(above, aprob)
    nmnn_auc = roc_auc_score(nnormal, nnprob)
    nmb_auc = roc_auc_score(below, bprob)
    
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(above, ns_probs)
    nma_fpr, nma_tpr, _ = roc_curve(above, aprob)
    nmnn_fpr, nmnn_tpr, _ = roc_curve(nnormal, nnprob)
    nmb_fpr, nmb_tpr, _ = roc_curve(below, bprob)
    
    # plot the ROC
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
    axes.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    axes.plot(nma_fpr, nma_tpr, color = 'r', label = 'Above normal, AUC = {:.3f}'.format(nma_auc))
    axes.plot(nmnn_fpr, nmnn_tpr, color = 'g', label = 'Near normal, AUC = {:.3f}'.format(nmnn_auc))
    axes.plot(nmb_fpr, nmb_tpr, color = 'y', label = 'Below normal, AUC = {:.3f}'.format(nmb_auc))
    axes.set_ylim(ymin = 0)
    axes.set_xlim(xmin = 0)
    axes.set(xlabel= 'False Positive Rate', ylabel='True Positive Rate', title = 'Receiver operating characteristic curve (ROC)')
    axes.legend(loc="upper left")
    
    del dataset['upper']
    del dataset['lower']
    
    return aprob, nnprob, bprob, above, nnormal, below

In [ ]:
# region1 = ROC_ML2020(forecast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2020-01-02', '2020-12-31')
# region2 = ROC_ML2020(forecast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2020-01-02', '2020-12-31')
NNl_roc3 = ROC_ML2020(forecast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2020-01-02', '2020-12-31')

In [ ]:
# Loading the saved models.
# Using the real-time forecasting of 2020 as input.
# adding the correlated inputs.

def CMte2020cor(region, spatial, model, lead_time):

    # import the data

    # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
    fecm = forecast_ecmwf.isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).chunk(dict(forecast_time=-1))
    hobs = hindcast_observation.sel(forecast_time = slice('2000-01-01', '2012-12-30')).isel(lead_time  = 0).chunk(dict(forecast_time=-1))
    hobs_int = hobs.interpolate_na(dim = 'forecast_time')
    hecm_int = hecm.interpolate_na(dim = 'forecast_time')
    fecm_interpolate = fecm.interpolate_na(dim = 'forecast_time')

    # modify the longitude and latitude to the selected regions:

    # regions

    region1 = hobs.sel(longitude = slice(95, 100)).sel(latitude = slice(36, 31))
    region2 = hobs.sel(longitude = slice(100, 105)).sel(latitude = slice(60, 55))
    region3 = hobs.sel(longitude = slice(5, 10)).sel(latitude = slice(53, 48))

    if region == 'region1':
        longitude = region1.longitude
        latitude  = region1.latitude

    if region == 'region2':
        longitude = region2.longitude
        latitude  = region2.latitude

    if region == 'region3':
        longitude = region3.longitude
        latitude  = region3.latitude

    # create the empty arrays for the results
    x = hecm_int.mean(dim = 'realization').sel(forecast_time = slice('2018-01-01', '2019-12-30'))

    prediction_an = np.zeros((53, len(latitude), len(longitude)))
    prediction_nn = np.zeros((53, len(latitude), len(longitude)))
    prediction_bn = np.zeros((53, len(latitude), len(longitude)))

    i = 0
    for long in longitude:

        j = 0
        for lat in latitude:

            stackX = None

            # choose the input and output data

            # hecm = hindcast_ecmwf.sel(forecast_time = slice('2000-01-01', '2019-12-30')).isel(lead_time = 0).mean(dim = 'realization')
            hecm_cell = hecm.sel(longitude = long, latitude = lat)
            hobs_cell = hobs.sel(longitude = long, latitude = lat)
            hobs_int = hobs_cell.interpolate_na(dim = 'forecast_time')
            hecm_int = hecm_cell.interpolate_na(dim = 'forecast_time')

            fecm_cell = fecm.sel(longitude = long, latitude = lat)
            fecm_int = fecm_cell.interpolate_na(dim = 'forecast_time')
            # hobs_int['cl'] = target_cl

            # operations
            # x inputs

            y = class_forecast.isel(lead_time = 0).sel(longitude = long, latitude = lat)
            yy = y.to_dataframe().reset_index()
            yy_inp = yy.drop(['latitude','longitude', 'lead_time'], axis = 'columns') # valid_time is not found in the case of using target_class
            yy_inpVar = yy_inp[['tp']]

            y_test = yy_inpVar.values.ravel()

            m = 1
            for r in range(0,11): 

                # training and testing data
                x = fecm_int.isel(realization = m)
                xx = x.to_dataframe().reset_index()
                xx['tp_lag1'] = xx.tp.shift(1).bfill()

                if spatial == 'one':
                    disla = lat - (1 * 1.5)
                    dislo = long - (1 * 1.5)
                    dla = np.arange(0, 4.5, 1.5)
                    dlo = np.arange(0, 4.5, 1.5)
                    v = 'var 4'

                if spatial == 'two':
                    disla = lat - (2 * 1.5)
                    dislo = long - (2 * 1.5)
                    dla = np.arange(0, 7.5, 1.5)
                    dlo = np.arange(0, 7.5, 1.5)
                    v = 'var 12'

                if spatial == 'three':
                    disla = lat - (3 * 1.5)
                    dislo = long - (3 * 1.5)
                    dla = np.arange(0, 10.5, 1.5)
                    dlo = np.arange(0, 10.5, 1.5)
                    v = 'var 24'

                # var = []
                nam = 0
                for ii in dlo:
                    for jj in dla:

                        xx['Var'] = fecm_interpolate.tp.sel(latitude = disla + jj, longitude = dislo + ii).isel(realization = m)
                        name = 'var {}'.format(nam)
                        xx.rename(columns={'Var': name}, inplace=True)
                        nam = nam + 1          

                xx_inp = xx.drop(['forecast_time','latitude','longitude', 'lead_time', 'valid_time', 'realization'], axis = 'columns')
                xx_inpVar = xx_inp.drop([v], axis = 'columns')

                x_test = xx_inpVar.values

                # Training the LR and MLP models

                min_max_scaler = preprocessing.MinMaxScaler()
                x_testT = min_max_scaler.fit_transform(x_test)

                m = m + 1
    #             if r == 9:
    #                 m = 50

                if  model == 'LR':
                    lr = LogisticRegression()
                    lr_model = lr.fit(x_trainT, y_train)
                    training_accuracy = lr_model.score(x_train, y_train)
    #                 print(training_accuracy)

                if model == 'MLP':

                    # load the model
                    Model = load_model('models/'+ region + spatial +'/model_' + str(lead_time) + str(r + 1) + '_' +  str(i) + str(j) + '.h5')

                    # make prediction
                    yhat = Model.predict(x_testT, verbose=0)

                    # stack predictions into [rows, members, probabilities]
                    if stackX is None:
                        stackX = yhat
                    else:
                        stackX = dstack((stackX, yhat))

            stackXX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
            stackXX = np.nan_to_num(stackXX)
            # Load the meta model

            meta_model = pickle.load(open('models/' + region + spatial + '/meta_model_' + str(lead_time) + '_' +  str(i) + str(j) + '.sav', 'rb'))

            yy_pred = meta_model.predict_proba(stackXX)

            # add the values to the empty array

            prediction_an[:, j, i] = yy_pred[:,2]
            prediction_nn[:, j, i] = yy_pred[:,1]
            prediction_bn[:, j, i] = yy_pred[:,0]

            j = j + 1 

            print(1)

        i = i + 1

    # Make an xarray of predictions data

    time_step = [i for i in range(0, len(yy_pred))]

    longitude = longitude.values
    latitude = latitude.values


    predict_an = xr.DataArray(
    data=prediction_an,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_an",
    ),
    )

    predict_nn = xr.DataArray(
    data=prediction_nn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_nn",
    ),
    )

    predict_bn = xr.DataArray(
    data=prediction_bn,
    dims=["time_step","latitude", "longitude"],
    coords=dict(
    longitude=(["longitude"], longitude),
    latitude=(["latitude"], latitude),      
    time_step= (["time_step"], time_step)
    ),
    attrs=dict(
    description="predictions_bn",
    ),
    )

    # make categories from the predictions

    categories = xr.concat([predict_bn, predict_nn, predict_an],'category').assign_coords(category=['below normal', 'near normal', 'above normal'])
    categorical_p = categories.rename({'time_step':'forecast_time'})
    
    return categorical_p

In [ ]:
categorical_p = CMte2020cor('region1', 'one' ,'MLP', 0)

In [ ]:
categorical_p['forecast_time'] = obs_p['forecast_time']

In [ ]:
RPSS_cl = RPSS_2020('region3', categorical_p, '2020-01-02', '2020-12-31')

In [ ]:
# region1 = ROC_ML2020(forecast_observation , categorical_p, tercile_edges, 100, 36, 'tp', 0, '2020-01-02', '2020-12-31')
# region2 = ROC_ML2020(forecast_observation , categorical_p, tercile_edges, 100, 55, 'tp', 0, '2020-01-02', '2020-12-31')
NNl_roc3 = ROC_ML2020(forecast_observation , categorical_p, tercile_edges, 6, 53, 'tp', 0, '2020-01-02', '2020-12-31')